In [1]:
%gui asyncio

In [2]:
import asyncio    
from ipylgbst import LegoBoostWidget,LedColor,Port, Sensor
from ipywidgets import Output
from IPython.display import display

# the robot's LED supports the following colors
colors = [
    LedColor.pink,
    LedColor.purple,
    LedColor.blue,
    LedColor.lightblue,
    LedColor.cyan,
    LedColor.green,
    LedColor.yellow,
    LedColor.orange,
    LedColor.red,
]

# Connect the device 
Note that connecting the device should take 5-10 sec

In [3]:
# number of concurrent "lanes"
n_lanes = 4 
boost = LegoBoostWidget(n_lanes=n_lanes)
boost.connect()
boost

Output()

LegoBoostWidget(n_lanes=4)

# Change Color of LED

In [ ]:
output = Output()
async def main(lane, log):

    for color in colors:
        log(f"set color to {color.value}")
        await lane.set_led_async(color)
        await asyncio.sleep(0.5)
        
boost.run_async_program(main, output=output);

# Drive a polygon like path

In [ ]:
speed = 50
d_forward  = 200
d_rotate = 300
output = Output()
async def main(lane,log):  
    for i in range(4):
        await lane.set_led_async("red")
        await lane.motor_angle_multi_async(angle=d_forward, power_a=speed,power_b=speed)
        await lane.set_led_async("green")
        await lane.motor_angle_multi_async(angle=d_rotate, power_a=speed,power_b=-1.0*speed)
        
boost.run_async_program(main, output=output);
output

# Buttons (sync but non blocking API)

the non-blocking sync API is the best for a "remote-controlled-rc-car" experience

In [ ]:
import ipywidgets as widgets

button_up = widgets.Button(description='Up')
button_down = widgets.Button(description='Down')
button_left = widgets.Button(description='Left')
button_right = widgets.Button(description='Right')
button_stop  = widgets.Button(description='Stop')

box_ud = widgets.VBox([button_up,button_stop,button_down])
box = widgets.HBox([button_left,box_ud,button_right])
output = Output()

def on_up(lane, log):
    # run sync but NON blocking
    lane.motor_time_multi(10, 50, 50)
    
def on_down(lane, log):
    # run sync but NON blocking
    lane.motor_time_multi(10, -50, -50)
    
def on_left(lane, log):
    # run sync but NON blocking
    lane.motor_time_multi(10, -50, 50)
    
def on_right(lane, log):
    # run sync but NON blocking
    lane.motor_time_multi(10, 50, -50)
    
def on_stop(lane, log):
    # run sync but NON blocking
    lane.motor_time_multi(0, 0, 0)
    
button_up.on_click(lambda _:    boost.run_program(on_up, output=output))
button_down.on_click(lambda _:  boost.run_program(on_down, output=output))
button_left.on_click(lambda _:  boost.run_program(on_left, output=output))
button_right.on_click(lambda _: boost.run_program(on_right, output=output))
button_stop.on_click(lambda _:  boost.run_program(on_stop, output=output))

display(box, output)

# Buttons (async api)

In [ ]:
import ipywidgets

button_disco = ipywidgets.Button(description='Disco')
button_motors_ab = ipywidgets.Button(description='Motors AB')
button_motor_c = ipywidgets.Button(description='Motor C')


box = ipywidgets.HBox([button_disco, button_motors_ab, button_motor_c])
output = Output()

async def disco(lane, log):
    button_disco.disabled = True
    for i in range(10):
        await lane.set_led_async(LedColor.pink)
        await lane.set_led_async(LedColor.blue)
    button_disco.disabled = False


async def motor_ab(lane, log):
    button_motors_ab.disabled = True
    await lane.motor_time_multi_async(seconds=2, power_a=10, power_b=10)
    await lane.motor_time_multi_async(seconds=2, power_a=-10, power_b=10)
    await lane.motor_time_multi_async(seconds=2, power_a=10, power_b=-10)
    await lane.motor_time_multi_async(seconds=2, power_a=-10, power_b=-10)
    button_motors_ab.disabled = False

async def motor_c(lane, log):
    button_motor_c.disabled = True
    await lane.motor_time_async(port=Port.D, seconds=2, power=10)
    await lane.motor_time_async(port=Port.D, seconds=2, power=-10)
    button_motor_c.disabled = False

# each program runs on its own lane st. they can run concurrent
button_disco.on_click(lambda _:     boost.run_async_program(disco, lane=0, output=output))
button_motors_ab.on_click(lambda _: boost.run_async_program(motor_ab, lane=1, output=output))
button_motor_c.on_click(lambda _:   boost.run_async_program(motor_c, lane=2, output=output))
display(box,output)

# Sensors

In [ ]:
has_bqplot = True
try:
    import bqplot
except ImportError:
    has_bqplot = False

if has_bqplot:
    from bqplot import pyplot as plt
    from bqplot import topo_load
    import numpy
    
    output = Output()
    
    duration = 10.0
    dt = 0.10
    clip_value = 200

    sensor_values = []
    time_points = []

    plt.figure()
    plt.plot([], [])
    plt.xlim(0,duration)
    plt.ylim(0, clip_value)
    plt.xlabel("time")
    plt.ylabel("distance sensor")
    plt.show()

    async def poll_distance_sensor(lane, log):
        global sensor_values
        global time_points
        t = 0.0
        while t < duration:
            time_points.append(t)
            d = numpy.nan_to_num(await lane.get_distance_async(), clip_value)
            d = numpy.clip(d,0, clip_value)
            sensor_values.append(d)
            await asyncio.sleep(dt)
            t += dt
            plt.plot(time_points, sensor_values)

    boost.run_async_program(poll_distance_sensor, output=output);
    display(output)

# Concurrent Programs

In [ ]:

async def set_leds(lane, log):

    for color in colors:
        log(f"set color to {color.value}")
        await lane.set_led_async(color)
        await asyncio.sleep(0.5)
        

speed = 26
d_forward  = 200
d_rotate = 300
async def drive(lane, output):  
    for i in range(4):
        await lane.motor_angle_multi_async(angle=d_forward, power_a=speed,power_b=speed)
        await lane.motor_angle_multi_async(angle=d_rotate, power_a=speed,power_b=-1.0*speed)

output = Output()
boost.run_async_programs_concurrently([set_leds,drive], output=output);